In [ ]:
#made on Google Colab, best results if checked there. All ResearchPapers and Stopword-List uploaded to google colab directory (click on the folder icon on the left)

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize , word_tokenize
import glob
import re
import os
import numpy as np
import sys


In [ ]:
f = open('Stopword-List.txt','r')
c = f.read()
f.close()
stopwords = c.split('\n')


In [ ]:
def tokenize(content):
    content = content.lower()
    content = content.replace("-", " ")
    content = content.replace("•"," ")
    content = content.translate(str.maketrans("", "", string.punctuation))
    words = re.split(r'\s+|\n+', content)
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return words

In [ ]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re

nltk.download('punkt')

# Function to preprocess a document
def preprocess_document(doc):
    # Tokenize the document
    words = tokenize(doc)
    # Remove stopwords and apply stemming
    stemmer = PorterStemmer()
    filtered_words = [stemmer.stem(word) for word in words if word.lower() not in stopwords]

    return filtered_words

# Function to clean text and generate positions
def clean_and_generate_positions(docs_directory):
    # Build inverted index
    inverted_index = {}

    # Loop through each document
    for filename in os.listdir(docs_directory):
        if filename.endswith(".txt"):
            # Read the content of the document
            with open(os.path.join(docs_directory, filename), 'r', encoding='ISO-8859-1') as file:
                document_content = file.read()
                document_content = re.sub(r'https?://(?:www\.)?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', document_content)  # remove urls
                document_content = re.sub(r'\S+\.com\b', '', document_content)  # remove .com
                document_content = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', document_content)  # remove emails
                document_content = re.sub(r'\b\d+\b', '', document_content)  # remove numbers
                document_content = re.sub(r'[^\w\s]', '', document_content)  # remove other useless punctuation
            # Preprocess the document
            processed_words = preprocess_document(document_content)
            # Update inverted index
            for position, word in enumerate(processed_words):
                if word not in inverted_index:
                    inverted_index[word] = []
                inverted_index[word].append((filename, position))

    return inverted_index

# Example usage
docs_directory = "/content/"
inverted_index = clean_and_generate_positions(docs_directory)
# print(inverted_index)

count = 0
for term in inverted_index:
  count+=1

print(count)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


17846


In [ ]:

stemmer = PorterStemmer()

def boolean_or(query_terms, inverted_index):
    result = set()

    for term in query_terms:
        stemmed_term = stemmer.stem(term)
        postings = set(inverted_index.get(stemmed_term, []))
        result.update(entry[0] for entry in postings)

    result = list(result)

    sorted_result = sorted(list(result), key=lambda x: int(x.split('.')[0]) if '.' in x else int(x))

    return sorted_result

# Example OR query
or_query = ['transformer', 'model']
result_or = boolean_or(or_query, inverted_index)

result_or = [doc.replace('.txt','') for doc in result_or]
# Print the result without "txt" extension
print(f"OR Query Result: {result_or}")


OR Query Result: ['1', '2', '3', '7', '8', '9', '12', '13', '14', '15', '16', '17', '18', '21', '22', '23', '24', '25', '26']


In [ ]:
def getposting(term,inverted_index):
  result = set()
  term = stemmer.stem(term)
  postings = set(inverted_index.get(term,[]))
  result.update(entry[0] for entry in postings)

  sorted_result = sorted(list(result), key=lambda x: int(x.split('.')[0]) if '.' in x else int(x))
  return sorted_result



In [ ]:

stemmer = PorterStemmer()

def boolean_and(query_terms, inverted_index):
    result = None
    common_docs = set()
    for term in query_terms:
        stemmed_term = stemmer.stem(term)
        postings = set(inverted_index.get(stemmed_term, []))
        if result is None:
            result = postings
        else:
            # Take the intersection of postings based on document name only
            common_docs = {entry[0] for entry in result}.intersection(entry[0] for entry in postings)
    # Convert the result to a sorted list by docID
    sorted_result = sorted(list(common_docs), key=lambda x: int(x.split('.')[0]) if '.' in x else int(x))

    return sorted_result

# Example AND query
and_query = ['feature', 'selection','redundancy']
result_and = boolean_and(and_query, inverted_index)

result_and = [doc.replace('.txt','') for doc in result_and]
# Print the sorted result
print(f" AND Query Result: {result_and}")


 AND Query Result: ['22', '23', '24', '25', '26']


In [ ]:

stemmer = PorterStemmer()
docs_directory = "/content/"

def boolean_not(not_query_terms, inverted_index):
    # Find documents containing the terms from the NOT query
    not_docs = set()
    result = set()

    for term in not_query_terms:
        stemmed_term = stemmer.stem(term)
        postings = set(inverted_index.get(stemmed_term, []))
        not_docs.update(entry[0] for entry in postings)



    for filename in os.listdir(docs_directory):
        if filename not in not_docs and filename.endswith(".txt") and filename != "Stopword-List.txt":
            result.add(filename)


    # Sort the result by docID
    sorted_result = sorted(result, key=lambda x: int(x.split('.')[0]) if '.' in x else int(x))
    return sorted_result

# Example NOT query
not_query = ['model']

result_not = boolean_not(not_query, inverted_index)

result_not = [doc.replace('.txt', '') for doc in result_not]
# Print the result without "txt" extension
print(f"NOT Query Result: {result_not}")


NOT Query Result: ['11']


In [ ]:
import string

def convertexpressiontopostfix(expression):
    # Using a stack to convert the user-entered query to a postfix expression
    precedence = {'(': 0, 'NOT': 1, 'AND': 2, 'OR': 3}
    stack = []
    output = []

    # Split the expression into words
    words = expression.split()

    for word in words:
        if word in ['NOT', 'AND', 'OR']:
            while stack and stack[-1] != '(' and precedence[word] < precedence[stack[-1]]:
                output.append(stack.pop())
            stack.append(word)
        elif word == '(':
            stack.append(word)
        elif word == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            if stack and stack[-1] == '(':
                stack.pop()
        else:
            output.append(word)

    while stack:
        output.append(stack.pop())

    # Join the elements with space to form a valid postfix expression
    return ' '.join(output)

def evaluatepostfixexpression(expression):
    print('postfix:', expression)
    # Evaluating the postfix expression using operators precedences
    stack = []
    operators = {'AND', 'OR', 'NOT'}
    precedence = {'NOT': 1, 'AND': 2, 'OR': 3}

    for word in expression.split(' '):
        if word not in operators:
            stack.append(word)
        else:
            if word == 'NOT':
                element = stack.pop()
                result = boolean_not([element], inverted_index)
            else:
                while stack and stack[-1] in operators and precedence[word] <= precedence[stack[-1]]:
                    operator = stack.pop()
                    if operator == 'NOT':
                        operand = stack.pop()
                        result = boolean_not([operand], inverted_index)
                    else:
                        operand2 = stack.pop()
                        operand1 = stack.pop()
                        if operator == 'AND':
                            result = boolean_and([operand1, operand2], inverted_index)
                        else:
                            result = boolean_or([operand1, operand2], inverted_index)
                    stack.append(result)
                stack.append(word)

    while stack:
        operator = stack.pop()
        if operator == 'NOT':
            operand = stack.pop()
            result = boolean_not([operand], inverted_index)
        else:
            operand2 = stack.pop()
            operand1 = stack.pop()
            if operator == 'AND':
                result = boolean_and([operand1, operand2], inverted_index)
            else:
                result = boolean_or([operand1, operand2], inverted_index)
        stack.append(result)

    return stack.pop()

def has_only_and(expression):
    operators = [operator for operator in expression.split() if operator in {'AND', 'OR', 'NOT'}]
    return all(operator == 'AND' for operator in operators)


def has_only_or(expression):
    operators = [operator for operator in expression.split() if operator in {'AND', 'OR', 'NOT'}]
    return all(operator == 'OR' for operator in operators)


def has_only_one_word(user_input):
    words = user_input.split()
    return len(words) == 1

def proximity_query(word1, word2, k, inverted_index):
    stemmed_word1 = stemmer.stem(word1)
    stemmed_word2 = stemmer.stem(word2)

    postings_word1 = inverted_index.get(stemmed_word1, [])
    postings_word2 = inverted_index.get(stemmed_word2, [])

    result = []

    for doc1, pos1 in postings_word1:
        for doc2, pos2 in postings_word2:
            if doc1 == doc2:
                # Check if there are any positions within the specified distance k
                if (abs(pos1 - pos2) == k):
                    result.append(doc1)

    return result


def is_proximity_query(user_input):
    # Check if the user input matches the format for a proximity query
    # Assuming the format is "word1 word2 /k" where k is an integer
    tokens = user_input.split()
    return len(tokens) == 3 and tokens[2].startswith('/')


while True:

  # Get user input for the query
  user_input = input("Enter your query: ")

  if is_proximity_query(user_input):
    # Extract information from the proximity query
    words, k = user_input.split()[:2], int(user_input.split('/')[1])
    # Call the proximity query function
    result = proximity_query(words[0], words[1], k, inverted_index)
    print("Documents where '{}' and '{}' are {} distance apart: {}".format(words[0], words[1], k, result))


  elif has_only_one_word(user_input):
    result = getposting(user_input,inverted_index)
    print(result)
  elif has_only_and(user_input):
    # Extract words and call boolean_and
    query_terms = user_input.split()
    query_terms = [term for term in query_terms if term != 'AND']
    print(query_terms)
    result = boolean_and(query_terms, inverted_index)
    print("Query Result:", result)

  elif has_only_or(user_input):
    # Extract words and call boolean_and
    query_terms = user_input.split()
    query_terms = [term for term in query_terms if term != 'OR']
    print(query_terms)
    result = boolean_or(query_terms, inverted_index)
    print("Query Result:", result)


  else:
    # Convert the infix expression to postfix
    postfix_expression = convertexpressiontopostfix(user_input)

    # Evaluate the postfix expression
    result = evaluatepostfixexpression(postfix_expression)

    # Print the result
    print("Query Result:", result)





Enter your query: heart AND metal
['heart', 'metal']
Query Result: []


KeyboardInterrupt: Interrupted by user